**Creating CATALOG, SCHEMA and TABLE**

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ctl_crypto;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_crypto.staging_schema;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.staging_schema.raw_silveR
USING DELTA
LOCATION "abfss://silver@storagecrypto18.dfs.core.windows.net/silver_crypto"

In [0]:
%sql
SELECT * FROM ctl_crypto.staging_schema.raw_silver
LIMIT 5;

**NORMALIZING THE GENERAL TABLE
**By extracting the fact and dimension tables**

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_crypto.gold_schema;

In [0]:
%sql
DROP TABLE IF EXISTS ctl_crypto.gold_schema.dim_roi;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_roi(
  roi_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING,
  roi_time DOUBLE,
  roi_currency STRING,
  roi_percentage DOUBLE,
  last_updated_ts TIMESTAMP 
)
USING DELTA
LOCATION "abfss://gold@storagecrypto18.dfs.core.windows.net/dim_roi_new";

In [0]:
%sql CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_crypto(
dim_crypto_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
id STRING,
name STRING,
symbol STRING,
ath DOUBLE,
ath_change_percentage DOUBLE,
market_cap_billion DOUBLE,
volume_to_market_cap_ratio DOUBLE,
last_updated_is TIMESTAMP
)
USING DELTA
LOCATION "abfss://gold@storagecrypto18.dfs.core.windows.net/dim_crypto";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.fact_crypto(
  id STRING PRIMARY KEY,
  dim_crypto_id BIGINT,
  dim_roi_id BIGINT,
  market_cap_rank BIGINT,
  max_supply DOUBLE,
  total_supply DOUBLE,
  circulating_supply DOUBLE,
  total_volume DOUBLE,
  market_cap DOUBLE,
  market_cap_change_24h DOUBLE,
  market_cap_change_percentage_24h DOUBLE,
  price_change_24h DOUBLE,
  price_change_percentage_24h DOUBLE,
  fully_diluted_valuation DOUBLE,
  high_24h DOUBLE,
  low_24h DOUBLE,
  current_price DOUBLE,
  data_partition DATE,
  last_updated_ts TIMESTAMP,
  FOREIGN KEY(dim_crypto_id) REFERENCES ctl_crypto.gold_schema.dim_crypto
  (dim_crypto_id),
  FOREIGN KEY(dim_roi_id) REFERENCES ctl_crypto.gold_schema.dim_roi(roi_id)
)
USING DELTA
LOCATION "abfss://gold@storagecrypto18.dfs.core.windows.net/fact_crypto";

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_roi AS target
USING (
    SELECT
        symbol AS id,
        roi_time,
        roi_currency,
        roi_percentage,
        ingested_time AS last_updated_ts
    FROM (
        SELECT
            symbol,
            roi_time,
            roi_currency,
            roi_percentage,
            ingested_time,
            ROW_NUMBER() OVER (PARTITION BY symbol ORDER BY ingested_time DESC) AS rn
        FROM ctl_crypto.staging_schema.raw_silver
    ) AS subquery
    WHERE rn = 1
) AS updated_rn
ON target.id = updated_rn.id
WHEN MATCHED THEN
UPDATE SET
    target.roi_time = updated_rn.roi_time,
    target.roi_currency = updated_rn.roi_currency,
    target.roi_percentage = updated_rn.roi_percentage,
    target.last_updated_ts = updated_rn.last_updated_ts
WHEN NOT MATCHED THEN
INSERT (
    id,
    roi_time,
    roi_currency,
    roi_percentage,
    last_updated_ts
)
VALUES (
    updated_rn.id,
    updated_rn.roi_time,
    updated_rn.roi_currency,
    updated_rn.roi_percentage,
    updated_rn.last_updated_ts
);

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.fact_crypto AS target
USING (
    SELECT
        symbol,
        market_cap_rank,
        CAST(max_supply AS DOUBLE) AS max_supply,
        CAST(total_supply AS DOUBLE) AS total_supply,
        CAST(circulating_supply AS DOUBLE) AS circulating_supply,
        CAST(total_volume AS DOUBLE) AS total_volume,
        CAST(market_cap AS DOUBLE) AS market_cap,
        CAST(market_cap_change_24h AS DOUBLE) AS market_cap_change_24h,
        CAST(market_cap_change_percentage_24h AS DOUBLE) AS market_cap_change_percentage_24h,
        CAST(price_change_24h AS DOUBLE) AS price_change_24h,
        CAST(price_change_percentage_24h AS DOUBLE) AS price_change_percentage_24h,
        CAST(fully_diluted_valuation AS DOUBLE) AS fully_diluted_valuation,
        CAST(high_24h AS DOUBLE) AS high_24h,
        CAST(low_24h AS DOUBLE) AS low_24h,
        CAST(current_price AS DOUBLE) AS current_price,
        data_partition,
        ingested_time AS last_updated_ts,
        ROW_NUMBER() OVER (PARTITION BY symbol ORDER BY ingested_time DESC) AS rn
    FROM ctl_crypto.staging_schema.raw_silver
) AS updated_rn
ON target.id = updated_rn.symbol AND updated_rn.rn = 1
WHEN MATCHED THEN
UPDATE SET
    target.market_cap_rank = updated_rn.market_cap_rank,
    target.max_supply = updated_rn.max_supply,
    target.total_supply = updated_rn.total_supply,
    target.circulating_supply = updated_rn.circulating_supply,
    target.total_volume = updated_rn.total_volume,
    target.market_cap = updated_rn.market_cap,
    target.market_cap_change_24h = updated_rn.market_cap_change_24h,
    target.market_cap_change_percentage_24h = updated_rn.market_cap_change_percentage_24h,
    target.price_change_24h = updated_rn.price_change_24h,
    target.price_change_percentage_24h = updated_rn.price_change_percentage_24h,
    target.fully_diluted_valuation = updated_rn.fully_diluted_valuation,
    target.high_24h = updated_rn.high_24h,
    target.low_24h = updated_rn.low_24h,
    target.current_price = updated_rn.current_price,
    target.data_partition = updated_rn.data_partition,
    target.last_updated_ts = updated_rn.last_updated_ts
WHEN NOT MATCHED THEN
INSERT (
    id,
    market_cap_rank,
    max_supply,
    total_supply,
    circulating_supply,
    total_volume,
    market_cap,
    market_cap_change_24h,
    market_cap_change_percentage_24h,
    price_change_24h,
    price_change_percentage_24h,
    fully_diluted_valuation,
    high_24h,
    low_24h,
    current_price,
    data_partition,
    last_updated_ts
)
VALUES (
    updated_rn.symbol,
    updated_rn.market_cap_rank,
    updated_rn.max_supply,
    updated_rn.total_supply,
    updated_rn.circulating_supply,
    updated_rn.total_volume,
    updated_rn.market_cap,
    updated_rn.market_cap_change_24h,
    updated_rn.market_cap_change_percentage_24h,
    updated_rn.price_change_24h,
    updated_rn.price_change_percentage_24h,
    updated_rn.fully_diluted_valuation,
    updated_rn.high_24h,
    updated_rn.low_24h,
    updated_rn.current_price,
    updated_rn.data_partition,
    updated_rn.last_updated_ts
);